In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/goodreads-books-reviews-290312/goodreads_test.csv
/kaggle/input/goodreads-books-reviews-290312/goodreads_sample_submission.csv
/kaggle/input/goodreads-books-reviews-290312/goodreads_train.csv


# STEP 1: Load the necessary Libraries

In [2]:
import pandas as pd
import csv
import string

# STEP 2: Load and explore the data

In [11]:
train = pd.read_csv("/kaggle/input/goodreads-books-reviews-290312/goodreads_train.csv")
test = pd.read_csv("/kaggle/input/goodreads-books-reviews-290312/goodreads_test.csv")

(train.shape,test.shape)

((900000, 11), (478033, 10))

# STEP 3: Sort and Separate data based on the labels

In [4]:
train.head()

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,8842281e1d1347389f2ab93d60773d4d,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb,5,This is a special book. It started slow for ab...,Sun Jul 30 07:44:10 -0700 2017,Wed Aug 30 00:00:26 -0700 2017,Sat Aug 26 12:05:52 -0700 2017,Tue Aug 15 13:23:18 -0700 2017,28,1
1,8842281e1d1347389f2ab93d60773d4d,16981,a5d2c3628987712d0e05c4f90798eb67,3,Recommended by Don Katz. Avail for free in Dec...,Mon Dec 05 10:46:44 -0800 2016,Wed Mar 22 11:37:04 -0700 2017,NaN,NaN,1,0
2,8842281e1d1347389f2ab93d60773d4d,28684704,2ede853b14dc4583f96cf5d120af636f,3,"A fun, fast paced science fiction thriller. I ...",Tue Nov 15 11:29:22 -0800 2016,Mon Mar 20 23:40:27 -0700 2017,Sat Mar 18 23:22:42 -0700 2017,Fri Mar 17 23:45:40 -0700 2017,22,0
3,8842281e1d1347389f2ab93d60773d4d,27161156,ced5675e55cd9d38a524743f5c40996e,0,Recommended reading to understand what is goin...,Wed Nov 09 17:37:04 -0800 2016,Wed Nov 09 17:38:20 -0800 2016,NaN,NaN,5,1
4,8842281e1d1347389f2ab93d60773d4d,25884323,332732725863131279a8e345b63ac33e,4,"I really enjoyed this book, and there is a lot...",Mon Apr 25 09:31:23 -0700 2016,Mon Apr 25 09:31:23 -0700 2016,Sun Jun 26 00:00:00 -0700 2016,Sat May 28 00:00:00 -0700 2016,9,1


In [5]:
train['rating'].value_counts()

4    313688
5    265007
3    188972
2     72627
0     30988
1     28718
Name: rating, dtype: int64

In [6]:
# Find out possible vals of the label or the parameter to be predicted
train['rating'].value_counts()

# Store all the sets of dataset with one label under one variable
train0=train[train['rating']==0]
train1=train[train['rating']==1]
train2=train[train['rating']==2]
train3=train[train['rating']==3]
train4=train[train['rating']==4]
train5=train[train['rating']==5]

# STEP 4: Reduction

In [7]:
# reducing each label data 1/100th times
train0=train0[:int(train0.shape[0]/100)]
train1=train1[:int(train1.shape[0]/100)]
train2=train2[:int(train2.shape[0]/100)]
train3=train3[:int(train3.shape[0]/100)]
train4=train4[:int(train4.shape[0]/100)]
train5=train5[:int(train5.shape[0]/100)]

# Checking the new dimensions of each label
train0.shape, train1.shape, train2.shape, train3.shape, train4.shape, train5.shape

((309, 11), (287, 11), (726, 11), (1889, 11), (3136, 11), (2650, 11))

# STEP 5: Combining data

In [8]:
# Merging
train=pd.concat([train0,train1,train2,train3,train4,train5],axis=0)

#Shuffling
train = train.sample(frac = 1)

In [9]:
train.head()

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
976,4672eb229c808b792b8ea95f01f19784,8490112,48c8723c4d2c6ecd68d849c67a4da38a,4,OH MY GOSH. I just finished the book and am st...,Mon Jun 25 13:00:49 -0700 2012,Sun May 25 14:07:04 -0700 2014,Sun May 25 14:07:04 -0700 2014,Fri May 16 00:00:00 -0700 2014,0,0
6745,1c845473e18c23f917126cb29bc8d243,32078787,637be5356c60f67f5d5e8f3654f9f6b9,4,My first Sarah Dessen book in many years I was...,Fri May 26 00:56:53 -0700 2017,Fri Jun 09 21:45:27 -0700 2017,Fri Jun 09 21:33:49 -0700 2017,Tue Jun 06 00:00:00 -0700 2017,1,0
10856,b11fb9f8b16ed8c8021fcc2a193a6016,13597755,959dc144a5803904f2ef62f6e2a9ac88,1,Ended up being a DNF for me. I would have prob...,Mon Jun 02 10:52:31 -0700 2014,Wed Sep 17 13:59:56 -0700 2014,NaN,Wed Sep 17 00:00:00 -0700 2014,0,0
6208,181d71656a197e41993c217703d4e748,11,f6713777eee30855a23d2c9557f9e048,5,"Just as hilarious, thought-provoking and charm...",Sun Sep 02 01:36:27 -0700 2012,Wed Nov 16 00:53:28 -0800 2016,Tue Jan 12 08:13:36 -0800 2016,Wed Dec 30 00:00:00 -0800 2015,0,0
7848,c4f655f2ed79599a9259d7762313f3f3,36638,9df90bd319eb1a1ebb1eeb69998a3656,4,I've been trying to get my kids to read this G...,Sat Jan 30 01:43:58 -0800 2016,Sat Jan 30 01:45:08 -0800 2016,Thu Jan 21 00:00:00 -0800 2016,NaN,1,0


In [10]:
train.shape

(8997, 11)